K-neighbors es lo recomendado para medir similaridad en datasets grandes por su eficiencia. Además, si se quiere extender la funcionalidad hacia user-item recommendation, es ideal también. En este modelo se puede definir distintas medidas de similaridad como distancia euclidiana o cosine similarity. 
Lo más importante es escoger correctamente los features a ingresar en la matriz, deben ser relevantes. -- Falta revisar si el método tfidf tiene un tope de palabras y si se puede asignar stopwords en español. 


-- Columnas a usar: genres, developer, tags, specs

# 1. Transformación del dataset para el modelo  

In [ ]:
import pandas as pd

games = pd.read_csv('datasets/steam_games.csv')

In [ ]:
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')
games['release_year'] = games['release_date'].dt.year
games['release_year'] = games['release_year'].fillna(0).astype(int)
games['release_year'] = games['release_year'].astype('str').replace('0', '')

In [ ]:
games.fillna('', inplace=True)

games.drop(games[games['title'] == ""].index, inplace=True)
games.shape

In [ ]:
# Aplica la función de unión con guión a la columna 'developer'
games['developer'] = games['developer'].str.replace(r'\s+', '-', regex=True)
games['developer'] = games['developer'].str.replace(r'---', "-")

In [ ]:
import re
import ast

# función de limpieza de las columnas
def tolist(lst):
    text= ""
    if isinstance(lst, str) and len(lst) > 0 and '[' in lst:
        lst = lst.lower()
        lst = lst.replace("0's",'0s').replace(';','').replace('&','').replace("'em","em")
        try:
            lst =  ast.literal_eval(lst)
        except (SyntaxError, ValueError):
            return lst.lower()
        
    if isinstance(lst,list):
        for item in lst:
            while "  " in item:
                item = item.replace("  "," ")
            item = item.replace(" ", "-")
            text = text + " " + item 
            text = text.replace('"','')
        return text.strip().lower()

    return lst

In [ ]:
# aplicamos la limpieza a cada columna 
games['specs'] = games['specs'].apply(lambda x : tolist(x))
games['genres'] = games['genres'].apply(lambda x : tolist(x))
games['tags'] = games['tags'].apply(lambda x : tolist(x))

In [ ]:
# Combine relevant columns into a single 'profile'
games['profile'] = games['genres'] + " " + games['tags'] + " " + games['specs'] + " " + games['developer'] + " " + games['release_year']

# Dataset listo para usarse en el modelo
games[['id','title','profile']].to_csv(r'./datasets/game_profile.csv',index=False)

# 2. Entrenamiento del modelo

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

df = pd.read_csv(r'./datasets/game_profile.csv')

# Create a TF-IDF Vectorizer to transform the 'profile' into numerical data
tfidf = TfidfVectorizer(stop_words='english',max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['profile'])

# Use k-NN to find similar games
knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # Using 6 neighbors to include the game itself
knn.fit(tfidf_matrix)

In [ ]:
# Crear función que crea las recomendaciones
def recommend(game_id):
    try:
        idx = df.index[df['id'] == game_id].tolist()[0]
        name = df['title'].loc[idx]
        # print(f'Porque te gusto "{name}" te recomendamos:')
        values , indices = knn.kneighbors(tfidf_matrix[idx])
        
        # Exclude the first (index 0) since it'll be the game itself
        game_indices = indices[0][1:]
        result = df['id'].iloc[game_indices]
    except:
        return [0]
    return list(result)

In [ ]:
# test
recommend(47810)

In [ ]:
# un df sin el profile
df1 = df[['id','title']]

In [ ]:
# función que crea la columna con las recomendaciones de cada juego
df1['recommends'] = df1['id'].apply(lambda x  : recommend(x))

In [ ]:
# Dataset para usar en la API listo 
df1['id'].fillna(0,inplace=True)
df1.to_csv(r'./dataquery/model_item.csv',index=False)
df1.info()

In [11]:
# Función del endpoint
import pandas as pd
import ast

def recomendacion_juego( game_id : int ):
    # carga de archivos
    df = pd.read_csv(r'./dataquery/model_item.csv')

    if game_id not in df['id'].values:
        return 'error'

    # Obtiene la lista de recomendaciones
    result = df[df['id'] == game_id]['recommends'].iloc[0]

    try:
        result = ast.literal_eval(result)
    except (SyntaxError, ValueError):
        print('No se puede convertir a lista')
        return 'error'

    response =[]
    item_buscado = df[df['id'] == game_id].iloc[0]
    
    for item_id in result:
        # Obtiene la información del juego recomendado
        item_info = df[df['id'] == item_id].iloc[0]  

        response.append({'game_id': int(item_info['id']), 
                         'title': item_info['title']})
    dict_respuesta = {'titulo_buscado': item_buscado['title'], 'results':response}  

    return dict_respuesta

In [12]:
# test
recomendacion_juego(47810)

{'titulo_buscado': 'Dragon Age: Origins - Ultimate Edition',
 'results': [{'game_id': 47730, 'title': 'Dragon Age™: Origins Awakening'},
  {'game_id': 17450, 'title': 'Dragon Age: Origins'},
  {'game_id': 17460, 'title': 'Mass Effect'},
  {'game_id': 7110, 'title': 'Jade Empire™: Special Edition'},
  {'game_id': 24980, 'title': 'Mass Effect 2'}]}